In [1]:
import os
import pandas as pd
import openpyxl as xl
import datetime

resource_dir = "../resources/"


def get_target_files(pre_fix):
    """Get the target files."""
    target_files = [file for file in os.listdir(resource_dir) if file.startswith(pre_fix)]
    return target_files


def concat_df(file_names):
    """Read in each csv file and concatenate into one dataframe."""
    all_dfs = [pd.read_csv(resource_dir + f) for f in file_names]
    return pd.concat(all_dfs)


def transform_data(df):
    """Some transformation."""
    sum_by_account = df.groupby("account")['amount'].sum().to_dict()
    return sum_by_account


def upadte_balancesheet(balancesheet, account_val_map):
    wb = xl.load_workbook(resource_dir + balancesheet)
    st = wb['BS'] 
    for cell in st["A"]:
        if cell.value in account_val_map:
            row_num = cell.row  # get row number
            # set the value for column B
            st["B" + str(row_num)].value = account_val_map[cell.value]
        
    # do the same thing with column E
    for cell in st["E"]:
        if cell.value in account_val_map:
            row_num = cell.row  # get row number
            # set the value for column F
            st["F" + str(row_num)].value = account_val_map[cell.value]
            
    return wb


def get_today():
    today = datetime.datetime.now()
    today_str = today.strftime("%Y-%m-%d")
    return today_str


def main():
    sources = get_target_files("bs")
    combined_df = concat_df(sources)
    account_map = transform_data(combined_df)
    bs_wb = upadte_balancesheet("balancesheet.xlsx", account_map)
    today_str = get_today()
    bs_wb.save(f"bs_{today_str}.xlsx")
    print("Done!!")
    

if __name__ == "__main__":
    main()

Done!!
